# 1- Loading datasets

In [ ]:
import pandas as pd

bpertussis_protein = pd.read_csv("./input/organismos/bpertussis/proteins_2_for_520_B_pertussis.csv")
bpertussis_peptides = pd.read_csv("./input/organismos/bpertussis/peptides_2_for_520_B_pertussis.csv")
bpertussis_taxonomy = pd.read_csv("./input/organismos/bpertussis/taxonomy_2_for_520_B_pertussis.csv")
bpertussis_taxonomy.head(10)

,ScientificName,Rank,UID,nPos,nNeg,nTot,IDs
0,Bacteria,superkingdom,2,2172,1833,4005,"208964,224914,1491,169963,623,632,1280,1773,27..."
1,Pseudomonadota,phylum,1224,551,276,827,"208964,224914,623,632,90370,562,784,573,243277..."
2,Betaproteobacteria,class,28216,117,42,159,"520,485,487,491,272560,28450,85698,72556,11722..."
3,Burkholderiales,order,80840,76,36,112,"520,272560,28450,85698,72556,95486"
4,Alcaligenaceae,family,506,63,34,97,"520,85698,72556"
5,Bordetella,genus,517,61,34,95,520
6,B. pertussis,organism,520,61,34,95,520


# 2 - Joining the datasets

In [ ]:
bpertussis_peptides_all_proteins = pd.merge(bpertussis_peptides, bpertussis_protein, on="Info_protein_id")

In [ ]:
bpertussis_peptides_all_proteins.head()

,Info_PepID,Info_organism_id_x,Info_host_id,Info_protein_id,Info_start_pos,Info_end_pos,Info_peptide,Info_peptide_length,Info_cluster_x,Info_split_x,Class,txid,Info_organism_id_y,Info_sequence,Info_cluster_y,Info_split_y
0,0511311A:4,562.0,10000000,0511311A,118,133,FGGDTAYSDDFFVGRV,16,269,split_03_20,1,562.0,562.0,AEIYNKDGNKVDLYGKAVGLHYFSKGNGQNSYGGDGDMTYARLGFK...,269,split_03_20
1,1007216A:2,1717.0,10000236,1007216A,369,383,QVVHNSYNRPAYSPG,15,57,split_04_20,1,1717.0,1717.0,GADDVVDSSKSFVMENFSSYHGTKPGYVDSIQKGIQKPKSGTQGNY...,57,split_04_20
2,1007216A:4,1717.0,10000000,1007216A,454,481,GRKIRMRCRAIDGDVTFCRPKSPVYVGN,28,57,split_04_20,1,1717.0,1717.0,GADDVVDSSKSFVMENFSSYHGTKPGYVDSIQKGIQKPKSGTQGNY...,57,split_04_20
3,1101217A:2,630.0,10000000,1101217A,14,30,SDWDCCDVCCNPACAGC,17,235,split_01_20,1,630.0,630.0,QACDPPSPPAEVSSDWDCCDVCCNPACAGC,235,split_01_20
4,1105030046:2,813.0,9606,1105030046,139,149,VINDLPLGRSI,11,83,split_03_20,1,813.0,813.0,MGSLVGRQAPDFSGKAVVCGEEKEISLADFRGKYVVLFFYPKDFTY...,83,split_03_20


In [ ]:
# consistency check
for index, row in bpertussis_peptides_all_proteins.iterrows():
    seq = row["Info_sequence"]
    start= row["Info_start_pos"] - 1
    end = row["Info_end_pos"]
    pepi = seq[start:end]
    if pepi != row["Info_peptide"]:
      print("Inconsistency")


In [ ]:
import re

lista_pseudomonadota = [208964,224914,623,632,90370,562,784,573,243277,666,287,670,470,727,520,192222,485,374932,83333,487,491,480,212042,672,177416,376619,272560,777,630,766147,738,948,624,770,32019,357244,197,320483,584,235279,585034,90371,155864,303,857571,645,85962,210,298,556,529507,945,944,205920,543,199310,214092,316401,269484,400667,28450,554,424717,102617,216597,571800,85698,72556,81475,122355,203192,316,1214225,1022,747,272620,955,89586,644,992121,1352357,1337393,1172206,674,1117322,29459,95486,392021,611,947,375177,267212,413502,28172,439184,866344]
lista_bpertussis = [520]

lista_pseudomonadota_minus_bpertussis = list(set(lista_pseudomonadota) - set(lista_bpertussis))

pseudomonadota_minus_bpertussis = bpertussis_peptides_all_proteins[bpertussis_peptides_all_proteins['txid'].isin(lista_pseudomonadota_minus_bpertussis)]
bpertussis = bpertussis_peptides_all_proteins[bpertussis_peptides_all_proteins['txid'].isin(lista_bpertussis)]

pseudomonadota_minus_bpertussis['Class'] = pseudomonadota_minus_bpertussis['Class'].apply(lambda x: 0 if x==-1 else x)
bpertussis['Class'] = bpertussis['Class'].apply(lambda x: 0 if x==-1 else x)


###pseudomonadota_minus_bpertussis

In [ ]:
groupby_Info_split = pseudomonadota_minus_bpertussis.groupby("Info_split_x")['Info_PepID'].count()
groupby_Info_split

,Info_PepID
Info_split_x,
split_01_20,118
split_02_20,158
split_03_20,160
split_04_20,142
split_05_20,154


In [ ]:
train_pseudomonadota_minus_bpertussis = pseudomonadota_minus_bpertussis[pseudomonadota_minus_bpertussis["Info_split_x"].isin(["split_01_20", "split_02_20", "split_04_20", "split_05_20"])]
test_pseudomonadota_minus_bpertussis = pseudomonadota_minus_bpertussis[pseudomonadota_minus_bpertussis["Info_split_x"].isin(["split_03_20"])]

train_pseudomonadota_minus_bpertussis.to_csv("./input/organismos/bpertussis/train_pseudomonadota_minus_bpertussis.csv", index=False)
test_pseudomonadota_minus_bpertussis.to_csv("./input/organismos/bpertussis/test_pseudomonadota_minus_bpertussis.csv", index=False)

base_path="./input/organismos/pseudomonadota_minus_bpertussis/"

for i in range(1, 6):
    fold_df = pseudomonadota_minus_bpertussis[pseudomonadota_minus_bpertussis["Info_split_x"].isin([f"split_0{i}_20"])]
    fold_df.to_csv(f"{base_path}folds/fold{i}.csv", index=False)

###bpertussis

In [ ]:
groupby_Info_split = bpertussis.groupby("Info_split_x")['Info_PepID'].count()
groupby_Info_split

,Info_PepID
Info_split_x,
split_01_20,49
split_02_20,24
split_03_20,5
split_04_20,8
split_05_20,9


In [ ]:
train_bpertussis = bpertussis[bpertussis["Info_split_x"].isin(["split_01_20", "split_02_20", "split_03_20", "split_04_20"])]
test_bpertussis = bpertussis[bpertussis["Info_split_x"].isin(["split_05_20"])]

# merge split_03_20 into split_01_20 due to the presence of only one class label in split_03_20
fold1 = bpertussis[bpertussis["Info_split_x"].isin(["split_01_20", "split_03_20"])]
fold2 = bpertussis[bpertussis["Info_split_x"].isin(["split_02_20"])]
#fold3 = bpertussis[bpertussis["Info_split_x"].isin(["split_03_20"])]
fold4 = bpertussis[bpertussis["Info_split_x"].isin(["split_04_20"])]
fold5 = bpertussis[bpertussis["Info_split_x"].isin(["split_05_20"])]

fold1.to_csv("./input/organismos/bpertussis/folds/fold1.csv", index=False)
fold2.to_csv("./input/organismos/bpertussis/folds/fold2.csv", index=False)
#fold3.to_csv("./input/organismos/bpertussis/folds/fold3.csv", index=False)
fold4.to_csv("./input/organismos/bpertussis/folds/fold4.csv", index=False)
fold5.to_csv("./input/organismos/bpertussis/folds/fold5.csv", index=False)
